# Deployment to Snowpark Container Service Demo

### Snowflake-ML-Python Installation

- Please refer to our [landing page](https://docs.snowflake.com/en/developer-guide/snowpark-ml/index) to install `snowflake-ml-python`.

## Train a model with Snowpark ML API 

In [1]:
from typing import Tuple
from snowflake.ml.modeling import linear_model
from sklearn import datasets
import pandas as pd
import numpy as np

def prepare_logistic_model() -> Tuple[linear_model.LogisticRegression, pd.DataFrame]:
    iris = datasets.load_iris()
    df = pd.DataFrame(data=np.c_[iris["data"], iris["target"]], columns=iris["feature_names"] + ["target"])
    df.columns = [s.replace(" (CM)", "").replace(" ", "") for s in df.columns.str.upper()]

    input_cols = ["SEPALLENGTH", "SEPALWIDTH", "PETALLENGTH", "PETALWIDTH"]
    label_cols = "TARGET"
    output_cols = "PREDICTED_TARGET"

    estimator = linear_model.LogisticRegression(
        input_cols=input_cols, output_cols=output_cols, label_cols=label_cols, random_state=0, max_iter=1000
    ).fit(df)

    return estimator, df.drop(columns=label_cols).head(10)

## Start Snowpark Session

In [2]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

session = Session.builder.configs(SnowflakeLoginOptions()).create()


SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


In [3]:
from snowflake.ml.registry import model_registry
from snowflake.ml._internal.utils import identifier

db = identifier._get_unescaped_name(session.get_current_database())
schema = identifier._get_unescaped_name(session.get_current_schema())

# will be a no-op if registry already exists
model_registry.create_model_registry(session=session, database_name=db, schema_name=schema) 
registry = model_registry.ModelRegistry(session=session, database_name=db, schema_name=schema)

## Register SnowML Model

In [4]:
logistic_model, test_features = prepare_logistic_model()
model_name = "snowpark_ml_logistic"
model_version = "v1"

model_ref = registry.log_model(
    model_name=model_name,
    model_version=model_version,
    model=logistic_model,
    sample_input_data=test_features,
)

"""
If your model has been logged and you want to reference it, you can use:
model_ref = model_registry.ModelReference(
    registry=registry, model_name=model_name, model_version=model_version
)
"""

/Users/shchen/micromamba/envs/snowml_1.0.12/lib/python3.10/site-packages/snowflake/ml/_internal/env_utils.py:217: UserWarning: Package requirement snowflake-snowpark-python<2,>=1.8.0 specified, while version 1.6.1 is installed. Local version will be ignored to conform to package requirement.
  warnings.warn(
/Users/shchen/micromamba/envs/snowml_1.0.12/lib/python3.10/site-packages/snowflake/ml/_internal/env_utils.py:217: UserWarning: Package requirement snowflake-snowpark-python<2,>=1.8.0 specified, while version 1.6.1 is installed. Local version will be ignored to conform to package requirement.
  warnings.warn(


'\nIf your model has been logged and you want to reference it, you can use:\nmodel_ref = model_registry.ModelReference(\n    registry=registry, model_name=model_name, model_version=model_version\n)\n'

## Model Deployment to Snowpark Container Service

In [5]:
# Optionally enable INFO log level to show more logging during model deployment.
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [6]:
from snowflake.ml.model import deploy_platforms
from snowflake import snowpark

compute_pool = "DEV_INFERENCE_CPU_POOL" # Pre-created compute pool
deployment_name = "LOGISTIC_FUNC" # Name of the resulting UDF

deployment_info = model_ref.deploy(
    deployment_name=deployment_name, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    target_method="predict",
    options={
        "compute_pool": compute_pool,
        "enable_ingress": True,
        #num_gpus: 1 # Specify the number of GPUs for GPU inferenc
    }
)

deployment_info

INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN SHULIN_DB.SHULIN_SCHEMA]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM SHULIN_DB.SHULIN_SCHEMA._SYSTEM_REGISTRY...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [CREATE STAGE IF NOT EXISTS SHULIN_DB.SHULIN_SCHEMA._SYSTEM_REGISTRY_DEPLOYMENTS_...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM SHULIN_DB.SHULIN_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: 

{'name': 'SHULIN_DB.SHULIN_SCHEMA.LOGISTIC_FUNC',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'predict',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.DOUBLE, name='SEPALLENGTH'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='SEPALWIDTH'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='PETALLENGTH'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='PETALWIDTH')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.DOUBLE, name='SEPALLENGTH'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='SEPALWIDTH'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='PETALLENGTH'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='PETALWIDTH'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='PREDICTED_TARGET')
                     ]
                 ),
 'options': {'compute_pool': 'DEV_INFERENCE_CPU_POOL', 'enable_ingress': True},
 'details': {'servi

In [7]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

## Batch Prediction on Snowpark Container Service

In [8]:
model_ref.predict(deployment_name, test_features)

,SEPALLENGTH,SEPALWIDTH,PETALLENGTH,PETALWIDTH,PREDICTED_TARGET
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


## Invoke Service Public Endpoint on Snowpark Container Service

### Prerequisites:
- For Limited Private Preview, the ACCOUNTADMIN of your Snowflake account must execute the following command:
```
CREATE SECURITY INTEGRATION SNOWSERVICES_INGRESS_OAUTH
TYPE=oauth
OAUTH_CLIENT=snowservices_ingress
ENABLED=true;
```

### Notes:
- Because Snowpark Containers uses Snowflake OAuth to enable ingress, the default role of the user cannot be any of the privileged roles, including ACCOUNTADMIN, SECURITYADMIN, and ORGADMIN. For more information, see Blocking Specific Roles from Using the Integration.

- Not everyone can access the public endpoints a service exposes. Only users in the same Snowflake account having a role with USAGE privilege on a service can access the public endpoints of the service.

For more details, please refers to https://docs.snowflake.com/LIMITEDACCESS/snowpark-containers/working-with-services#ingress-using-a-service-from-outside-snowflake for detailed setup to enable public endpoint on Snowpark Container Service.





In [9]:
import json
def get_service_endpoint():
    service_info = deployment_info["details"]["service_info"]
    rows = session.sql(f'DESCRIBE SERVICE {service_info["database_name"]}.{service_info["schema_name"]}.{service_info["name"]}').collect()
    res = rows[0]["public_endpoints"]
    if "provisioning in progress" in res:
        raise Valuere("Endpoints provisioning in progress. Please retry in a few seconds")        
    res_json = json.loads(res)
    target_method = deployment_info["target_method"]
    return res_json[target_method]

def get_session_token(session) -> str:
    """
    Gets session token from Snowflake client.
    """
    return session._conn._conn._rest._token_request("ISSUE")["data"]["sessionToken"]

In [10]:
data = {"data": [[index, {"_ID": index, **row.to_dict()}] for index, row in test_features.iterrows()]}

In [11]:
import requests

# Temporarily reset PYTHON_CONNECTOR_QUERY_RESULT_FORMAT needed for obtaining session token. 
session.sql("ALTER SESSION SET PYTHON_CONNECTOR_QUERY_RESULT_FORMAT = 'json'").collect()

session_token = get_session_token(session)
headers = {
    "Authorization": f'Snowflake Token="{session_token}"',
}


api_endpoint = f"https://{get_service_endpoint()}/predict"

res = requests.post(api_endpoint, json=data, headers=headers)

session.sql("ALTER SESSION SET PYTHON_CONNECTOR_QUERY_RESULT_FORMAT = 'arrow'").collect()

res.json()["da"]


NameError: name 'Valuere' is not defined

## Cleanup 

In [ ]:
model_ref.delete_deployment(deployment_name=deployment_name)

In [ ]:
model_ref.delete_model()